<a href="https://colab.research.google.com/github/jazkre/01RAD/blob/main/assignments/2022/assignement03/01RAD2022_assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. zápočtová úloha z 01RAD

## Popis úlohy

Datový soubor vychází z datasetu \texttt{House Sales in King County, USA}, který je k nalezení například na kaggle.com, nebo v knihovně \texttt{library(moderndive)} data \texttt{house\_prices}.
Původní dataset obsahuje prodejní ceny domů v oblasti King County, která obsahuje i město Seattle, a data byla nasbírána mezi květnem 2014 a květnem 2015. Pro naše potřeby bylo z datasetu vypuštěno jak několik proměnných, také byl dataset výrazně osekán a lehce modifikován.

Dále byl dataset již dopředu rozdělen na tři části, které všechny postupně v rámci 3. zápočtové úlohy využijete.


In [ ]:
library(tidyverse)

In [ ]:
knitr::opts_chunk$set(echo = FALSE)
list_of_packages <- c("tidyverse", "MASS","GGally","moderndive", "dplyr","car","lmtest","nortest","psych","ggpubr","corrplot","RColorBrewer")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
fig <- function(width, heigth){
 options(repr.plot.width = width, repr.plot.height = heigth)
 }
 options(jupyter.plot_scale=2)

In [ ]:
house_rad <- read.table("https://raw.githubusercontent.com/francji1/01RAD/main/data/01RAD2022_house.csv",header=TRUE,sep=",")
head(house_rad)

Data celkem obsahují  následujících 18 proměnných, přičemž naším cílem je prozkoumat vliv 12 z nich na cenu nemovitostí   * price}. Přičemž anglický popis jednotlivých proměnných (sloupců) je následující:

Feature & Description
 
  * id  : Our notation for a house  
 
  * price  : Price is prediction target  
 
  * bedrooms  : Number of Bedrooms/House  
 
  * bathrooms  : Number of Bathrooms/Bedrooms  
 
  * sqft\_living  : Square footage of the home  
 
  * sqft\_lot  : Square footage of the lot   
 
  * floors  :  Total floors (levels) in house 
 
  * waterfront  : House which has a view to a waterfront   
 
  * view  : Has been viewed  
 
  * condition  : How good the condition is Overall  
 
  * grade  : Overall grade given to the housing unit 
 
  * sqft\_above  : Square footage of house apart from basement   
 
  * sqft\_basement  : Square footage of the basement   
 
  * yr\_built  : Built Year   
 
  * yr\_renovated  : Year when house was renovated  
 
  * sqft\_living15  : Living room area in 2015 (implies– some renovations) 
 
  * sqft\_lot15  : lotSize area in 2015 (implies– some renovations)  
 
  * split  : Splitting variable with train, test and validation sample  


## Podmínky a body

Úkol i protokol vypracujte samostatně. Pokud na řešení nějaké úlohy budete přesto s někým spolupracovat, radit se, nezapomeňte to u odpovědi na danou otázku uvést. Tato zápočtová úloha obsahuje 10 otázek po 1 bodu. Celkem za 3 zápočtové úlohy bude možné získat 30 bodů, přičemž pro získání zápočtu je potřeba 20. Další dodatečné body mohu případně individuálně udělit za řešení mini domácích úkolů z jednotlivých hodin.

## Odevzdání

Protokol ve formátu pdf (včetně příslušného Rmd souboru) odevzdejte prostřednictvím MS Teams, nejpozději do 12:00 dne 5. 1. 2023.


## Průzkumová a grafická část:

* Otázka 01

  Ověřte rozměry datového souboru, typy jednotlivých proměnných,  a shrňte základní popisné charakteristiky všech proměnných. Vykreslete histogram a odhad hustoty pro odezvu   * price}, dá se z toho již něco odvozovat pro budoucí analýzu?
  
  \

Dataset obsahuje 2000 objektov (pozorovaní) s 19 premennými. 

Typy jednotlivých premenných nám poskytuje náhľad `head()`, respektíve výčet pomocou parametru *class*. Čo sa týka klasifikácie premenných, za *kvalitatívne* považujeme `waterfront`, `condition`, `grade` plus ako *ordinálne* chápeme `X` a `id` a značia poradie objektu, identifikátor. Ostatné premenná ako `price`, `bedrooms`, `bathrooms`, `sqft_living` a ostatné doposiaľ nevymenované považujeme za *kvantitatívne*.

Základné štatistiky uvádzame vrámci výpisu funkcie `summary()`.

In [ ]:
head(house_rad)

In [ ]:
print("Rozmery dátového súboru: ")
dim(house_rad)
print("")
print("Typy jednotlivých premenných: ")
sapply(house_rad, class)

In [ ]:
unique(house_rad$waterfront)
unique(house_rad$view)

In [ ]:
#
summary(house_rad)

Premenná `X` je totožná s `id`, preto ju vynecháme. Máme teda **2000** pozorovaní s **18** premennými. 

In [ ]:
house_rad<- house_rad[,!names(house_rad) %in% c("X")]

In [ ]:
attach(house_rad)
pairs.panels(house_rad,smooth=F,lm=T,scale=T,pch=20)

**Pozn:** nie je uvedené, ale predpokladáme cenu v amerických dolároch.

Zostrojíme histogram a odhad hustoty pre odozvu `price`. Vidíme, že v datasete máme niekoľko odľahlých pozorovaní, tj. veľmi vysoké ceny domov. 11 pozorovaní má cenu vyššiu než *2 900 000*, pričom keď si ich vypíšeme, vidíme, že dom s id=500 má chybne uvedenú obytný plochu 179 namiesto 1790 a vysokú hodnotu grade, ktorá pravdepodobne závisí na obytnej ploche, ale vzťah nám nie je známy, aby sme príslušnú hodnotu mohli prípadne opraviť. Niekoľko domov v tejto kategórií má veľkú plochu pozemku. Dom s id=100 má uvedenú cenu *8 500 000* a rok výstavby je 1911. Keď si vypíšeme domy postavené okolo tohto dátumu, s prihliadnutím na počet izieb, pohybujeme sa v cenovej kategórií desaťnásobne nižších súm. 

Na základe týchto postrehov by sme mali byť pri práci s týmito nehnuteľnosťami opatrný, dokonca je na mieste uvažovať možnosť modelu pre cenovo nižšie kategórie domov zvášť.

 Odhadnutá hustota naznačuje *Gaussovo normálne rozdelenie*.

In [ ]:
fig(10,6)
ggplot(data = house_rad, aes(price))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  labs(title="Histogram") +
  labs(x= "Cena ", y="hustota")

In [ ]:
exps=subset(house_rad, house_rad$price>2900000)
dim(exps)
exps

In [ ]:
subset(house_rad, house_rad$yr_built<1915 & house_rad$bedrooms<4)

In [ ]:
unique(house_rad$grade)
unique(house_rad$condition)
unique(house_rad$floors)

In [ ]:
subset(house_rad,house_rad$bedrooms==3 & house_rad$sqft_living>=1300 & house_rad$sqft_living<2000 & house_rad$yr_built>2005)

In [ ]:
subset(house_rad,house_rad$yr_renovated==0 & house_rad$sqft_lot!=house_rad$sqft_lot15 & house_rad$sqft_living!=house_rad$sqft_living15)

* Otázka 02

  Jsou všechny proměnné použitelné pro analýzu a predikci ceny nemovitostí? Pokud data obsahují chybějící hodnoty, (případně nesmyslné hodnoty), lze je nějak nahradit (upravit), nebo musíme data odstranit?
  
  \

Z funkcie `summary()` vidíme, že dataset neobsahuje chýbajúce hodnoty. Premennú `X` sme už odstránili (bola totožná s premennou `id`), premennú `id` vyslovene nepotrebuje, ale zatiaľ ju ponecháme ako identifikátor domu (aspoň pre potreby tejto úlohy), na ktorý sa budeme odkazovať.

Už v predošlej úlohe sme načali podozrivé hodnoty. Všimli sme si:


*   dom s id=500 má chybne uvedenú obytný plochu 179 namiesto 1790 a vysokú hodnotu grade (174), ktorá pravdepodobne závisí na obytnej ploche, ale vzťah nám nie je zo zadania známy, aby sme príslušnú hodnotu mohli prípadne opraviť. Toto pozorovanie odstránime z datasetu. V praxi by sme túto úpravu naviac diskutovali s poskytovateľom datasetu.


*   dom s id=100 má uvedenú cenu *8 500 000* a rok výstavby je 1911. Keď si vypíšeme domy postavené okolo tohto dátumu, s prihliadnutím na počet izieb okolo 3, pohybujeme sa v cenovej kategórií desaťnásobne nižších súm. V inom ohľade sa tento dom nelíši pre zvolený výber. Stálo by za zváženie upraviť cenu na *850 000*, v praxi by sme túto úpravu diskutovali s poskytovateľom datasetu.


* obdobný problém majú aj domy s id=200 a 300, kedy je cena 10násobne vyššia bez akéhokoľvek iného viditeľného dôvodu. (Nemožno vylúčiť, že sa v dome narodila/žila/zomrela významná osobnosť)

$⟶$ pozorovania 100, 200, 300 vylúčime z datasetu (stálo by za pokus vydeliť cenu 10 a pracovať s takto upraveným datasetom, ale takýto postup bez konzultácie s poskytovateľom datasetu, by nás mohol stáť napr. pozíciu rektora niekde v na severe Viedne 😀)

*   v datasete sa nachádza celkovo 6 pozorovaní vrátane domu 500 (ďalej 600,700,800,900,1000) s rovnakým problémom ako v bode 1: trojmiestna hodnota `grade` a obytná plocha zle zadaná - chýba 0. Tieto pozorovania vymažeme, lebo nevieme opraviť premennú `grade`


*   posledným podozrivým úkazom je uvedený rok renovácie 0 (teda že dom nebol renovovaný) ale dochádza k zmene plochy domu či pozemku k roku 2015. K datasetu nie je poskytnutá bližšia informácia o rozsahu renovácií obecne a kedy je nutné ich zaznamenať, či dokonca informácie o prikúpení pozemku. S týmto postrehom teda nebudeme robiť zmeny.





In [ ]:
subset(house_rad,house_rad$sqft_lot<house_rad$sqft_lot15 & house_rad$yr_renovated==0)

In [ ]:
subset(house_rad,house_rad$grade>99)


In [ ]:
subset(house_rad,house_rad$sqft_living!=(house_rad$sqft_above+house_rad$sqft_basement))

In [ ]:
subset(house_rad,house_rad$sqft_lot>11000)

In [ ]:
houses_new=subset(house_rad, house_rad$grade<99)
houses_new=subset(houses_new, houses_new$id!=100 & houses_new$id!=200 & houses_new$id!=300)

In [ ]:
unique(houses_new$grade)

In [ ]:
data_Brno=subset(house_rad, house_rad$grade<99)

In [ ]:
for (idx in c(100,200,300)){
  data_Brno$price[data_Brno$id==idx ]<-data_Brno$price[data_Brno$id==idx ]/10
}
#data_Brno$price[data_Brno$id==100 ]<-data_Brno$price[data_Brno$id==100 ]/10
#data_Brno$price[data_Brno$id==200 ]<-data_Brno$price[data_Brno$id==200 ]/10
#data_Brno$price[data_Brno$id==300 ]<-data_Brno$price[data_Brno$id==300 ]/10

In [ ]:
#subset(data_Brno, data_Brno$id==100 | data_Brno$id==200 | data_Brno$id==300)

In [ ]:
#dim(houses_new)

Po eliminácií niektorých pozorovaní dostávame dataset s 1991 objektami.

* Otázka 03
 
  Zkontrolujte pro 4 vybranné proměnné (price, sqft_living, grade, yr_built) bylo-li rozdělení datasetu pomocí proměnné split náhodné. Tj mají zmíněné proměnné ve skupinách train, test a validation přibližně stejné rozdělení?

\

Rozdelíme si dáta na 3 podmnožiny: *trénovaciu, testovaciu* a *validačnú*. *Trénovacia* množina má 991 objektov, čo je takmer dvojnásobok než majú *testovacia* a *validačná*, ktoré majú každá po 500. (Čo však dáva zmysel z hľadiska machine learningu).

In [ ]:
h_test<-subset(houses_new,houses_new$split=="test")
h_train<-subset(houses_new,houses_new$split=="train")
h_validation<-subset(houses_new,houses_new$split=="validation")

In [ ]:
dim(h_train)
dim(h_validation)
dim(h_test)

Vykreslíme si príslušné histogramy, ktoré medzi sebou porovnáme naprieč 3 split-skupinami. Pre každú premennú vykreslíme 3 histogramy s rovnakým nastaveným počtom binov a mierkou.

Na jednotlivých skupinách histogramov možno vidieť, že 3 skupiny sú si relatívne podobné. Pre premennú `price` sa pomerne zhoduje telo rozdelenia a rozdiely sú badateľné v pravom chvoste. Pre premennú `sqft_living` sedí rovnaký trend, rovnako aj vrámci premennej `grade`, kde sa trénovacie dáta líšia o niečo viac. Najmarkantnejšie rozdiely pozorujeme vrámci premennej `yr_built`, kde rozdelenia sedia viac na chvostoch než v tele rozdelenia. 

V konenčnom dôsledku považujeme rozdelenie skupín za postačujúce.

In [ ]:
fig(17,9)
hhp<-ggplot(houses_new, aes(x = price, fill = split,col=split)) + 
  geom_histogram(aes(y=..density..),position = "identity", alpha = 0.2, bins = 26) +
  scale_fill_manual(values=c("firebrick1", "skyblue1","darkgoldenrod1")) +
  scale_color_manual(values=c("firebrick4", "dodgerblue4","darkgoldenrod3")) +
  theme_light() +
    xlim(0,max(houses_new$price)*1.1) +
  geom_line(stat = "density")+
  labs(x= "Cena ", y="hustota")
hhs<-ggplot(houses_new, aes(x = sqft_living, fill = split,col=split)) + 
  geom_histogram(aes(y=..density..),position = "identity", alpha = 0.2, bins = 26) +
  scale_fill_manual(values=c("firebrick1", "skyblue1","darkgoldenrod1")) +
  scale_color_manual(values=c("firebrick4", "dodgerblue4","darkgoldenrod3")) +
  theme_light() +
    xlim(0,max(houses_new$sqft_living)*1.1) +
  geom_line(stat = "density")+
  labs(x= "Plocha ", y="hustota")
hhg<-ggplot(houses_new, aes(x = grade, fill = split,col=split)) + 
  geom_histogram(aes(y=..density..),position = "identity", alpha = 0.2, bins = 26) +
  scale_fill_manual(values=c("firebrick1", "skyblue1","darkgoldenrod1")) +
  scale_color_manual(values=c("firebrick4", "dodgerblue4","darkgoldenrod3")) +
  theme_light() +
    xlim(3,max(houses_new$grade)+1.1) +
  geom_line(stat = "density")+
  labs(x= "Grade ", y="hustota")
hhy<-ggplot(houses_new, aes(x = yr_built, fill = split,col=split)) + 
  geom_histogram(aes(y=..density..),position = "identity", alpha = 0.2, bins = 26) +
  scale_fill_manual(values=c("firebrick1", "skyblue1","darkgoldenrod1")) +
  scale_color_manual(values=c("firebrick4", "dodgerblue4","darkgoldenrod3")) +
  theme_light() +
    xlim(min(houses_new$yr_built)-3,max(houses_new$yr_built)+3)+
  geom_line(stat = "density")+
  labs(x= "Rok výstavby ", y="hustota")

ggarrange(hhp,hhs,hhg,hhy, ncol = 2, nrow = 2, common.legend = FALSE, legend = "bottom")

In [ ]:
fig(17,9)
hp<-ggplot(data = houses_new, aes(price))+
  geom_histogram(aes(y=..density..),data = h_train, col="firebrick4", fill="firebrick1", alpha = .3,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_test, col="dodgerblue4", fill="steelblue1", alpha = .2,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_validation, col="darkgoldenrod3", fill="darkgoldenrod1", alpha = .1,bins =26) + 
  theme_light() +
    xlim(0,max(houses_new$price)*1.1) +
  geom_line(stat = "density")+
  labs(x= "Cena ", y="hustota")
hs<-ggplot(data = houses_new, aes(sqft_living))+
  geom_histogram(aes(y=..density..),data = h_train, col="firebrick4", fill="firebrick1", alpha = .3,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_test, col="dodgerblue4", fill="steelblue1", alpha = .2,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_validation, col="darkgoldenrod3", fill="darkgoldenrod1", alpha = .1,bins =26) + 
  theme_light() +
    xlim(0,max(houses_new$sqft_living)*1.1) +
  geom_line(stat = "density")+
  labs(x= "Plocha ", y="hustota")
hg<-ggplot(data = houses_new, aes(grade))+
  geom_histogram(aes(y=..density..),data = h_train, col="firebrick4", fill="firebrick1", alpha = .3,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_test, col="dodgerblue4", fill="steelblue1", alpha = .2,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_validation, col="darkgoldenrod3", fill="darkgoldenrod1", alpha = .1,bins =26) + 
  theme_light() +
    xlim(3,max(houses_new$grade)+1.1) +
  geom_line(stat = "density")+
  labs(x= "Grade ", y="hustota")
hy<-ggplot(data = houses_new, aes(yr_built))+
  geom_histogram(aes(y=..density..),data = h_train, col="firebrick4", fill="firebrick1", alpha = .3,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_test, col="dodgerblue4", fill="steelblue1", alpha = .2,bins =26) + 
  geom_histogram(aes(y=..density..),data = h_validation, col="darkgoldenrod3", fill="darkgoldenrod1", alpha = .1,bins =26) + 
  theme_light() +
    xlim(min(houses_new$yr_built)-3,max(houses_new$yr_built)+3)+
  geom_line(stat = "density")+
  labs(x= "Rok výstavby ", y="hustota")

#ggarrange(hp,hs,hg,hy, ncol = 2, nrow = 2, common.legend = FALSE, legend = "bottom")

In [ ]:
fig(17,9)

In [ ]:
h1<-ggplot(data = h_train, aes(price))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(0,max(houses_new$price)*1.1) +
  labs(title="Histogram-train data") +
  labs(x= "Cena ", y="hustota")

h2<-ggplot(data = h_test, aes(price))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(0,max(houses_new$price)*1.1)+
  labs(title="Histogram-test data") +
  labs(x= "Cena ", y="hustota")
h3<-ggplot(data = h_validation, aes(price))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(0,max(houses_new$price)*1.1)+
  labs(title="Histogram-validation data") +
  labs(x= "Cena ", y="hustota")

ggarrange(h1, h2, h3,hp ,ncol = 1, nrow = 4, common.legend = FALSE, legend = "bottom")

In [ ]:
h1<-ggplot(data = h_train, aes(sqft_living))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
theme_light() +
      xlim(0,max(houses_new$sqft_living)*1.1) +
  labs(title="Histogram-train data") +
  labs(x= "Plocha domu ", y="hustota")

h2<-ggplot(data = h_test, aes(sqft_living))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(0,max(houses_new$sqft_living)*1.1)+
  labs(title="Histogram-test data") +
  labs(x= "Plocha domu ", y="hustota")
h3<-ggplot(data = h_validation, aes(sqft_living))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(0,max(houses_new$sqft_living)*1.1)+
  labs(title="Histogram-validation data") +
  labs(x= "Plocha domu ", y="hustota")

ggarrange(h1, h2, h3,hs, ncol = 1, nrow = 4, common.legend = FALSE, legend = "bottom")

In [ ]:
h1<-ggplot(data = h_train, aes(grade))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
theme_light() +
      xlim(3,max(houses_new$grade)+1.1) +
  labs(title="Histogram-train data") +
  labs(x= "Grade ", y="hustota")

h2<-ggplot(data = h_test, aes(grade))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(3,max(houses_new$grade)+1.1)+
  labs(title="Histogram-test data") +
  labs(x= "Grade ", y="hustota")
h3<-ggplot(data = h_validation, aes(grade))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(3,max(houses_new$grade)+1.1)+
  labs(title="Histogram-validation data") +
  labs(x= "Grade ", y="hustota")

ggarrange(h1, h2, h3,hg, ncol = 1, nrow = 4, common.legend = FALSE, legend = "bottom")

In [ ]:
h1<-ggplot(data = h_train, aes(yr_built))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
theme_light() +
      xlim(min(houses_new$yr_built)-3,max(houses_new$yr_built)+3) +
  labs(title="Histogram-train data") +
  labs(x= "Rok výstavby ", y="hustota")

h2<-ggplot(data = h_test, aes(yr_built))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(min(houses_new$yr_built)-3,max(houses_new$yr_built)+3)+
  labs(title="Histogram-test data") +
  labs(x= "Rok výstavby ", y="hustota")
h3<-ggplot(data = h_validation, aes(yr_built))+
  geom_histogram(aes(y =after_stat(density)), col="plum4", fill="seashell1", alpha = .7,bins =26) + 
  geom_density(col = "steelblue1")+
  theme_light() +
    xlim(min(houses_new$yr_built)-3,max(houses_new$yr_built)+3)+
  labs(title="Histogram-validation data") +
  labs(x= "Rok výstavby ", y="hustota")

ggarrange(h1, h2, h3,hy, ncol = 1, nrow = 4, common.legend = FALSE, legend = "bottom")

## Lineární model (použijte pouze trénovací data, tj. split == "train"):

* Otázka 04

  Spočtěte korelace mezi jednotlivými regressory a graficky je znázorněte. Dále spočtěte číso podmmíněnosti matice regresorů Kappa a VIF. Pokud se v datech vyskytuje znatelná multicollinearita, rozhodněte jaké proměnné a proč použijete v následném linárním modelu.


Ďalej pracujeme iba s trénovacími dátami, teda môžeme odstrániť stĺpec premennej `split` a aj `id`, ktoré nám v tomto momente už neposkytujú žiadne valídne informácie. 

Touto voľbou sme dokonca obišli extrémne ceny domov a nemusíme sa zaoberať 2 modelmi. 

In [ ]:
h_tr_Brno<- subset(data_Brno, data_Brno$split=="train")
h_tr_Brno<- subset(h_tr_Brno, select = -c(split,id))

In [ ]:
h_train <- subset (h_train, select = -c(split,id)) 

In [ ]:
summary(h_train)
print("*********************************")
print("*********************************")
print("*********************************")
print("*********************************")
summary(h_tr_Brno)

Znázorníme maticu korelácie medzi jednotlivými regresormi. Najvyššie hodnoty dosahujú dvojice `sqft_living` a `sqft_above` 0.87, čo je logické, pretože plocha obytná sa dá rozdeliť na plochu nadzemia a plochu povnice. Ďalej dvojice `sqft_lot` a `sqft_lot15` 0.76, `bathrooms` a `sqft_living` 0.74, `sqft_living` a `sqft_living15`, čo sa dalo očakávať.

Ďalej významné sú dvojice: `price` a `sqft_living` 0.67, `price` a `grade` 0.68, `sqft_living` a `grade` 0.72 (túto závislosť sme odtušili v otázke 1).



In [ ]:
# improved correlation matrix

corrplot(cor(h_train),
  method = "color",
  type = "upper", # show only upper side
   col=brewer.pal(n=8, name="PuOr"),
   addCoef.col = "black", # Add coefficient of correlation
         tl.col="black", tl.srt=45 #Text label color and rotation
         # Combine with significance
)


In [ ]:
corrplot(cor(h_tr_Brno),
  method = "color",
  type = "upper", # show only upper side
   col=brewer.pal(n=8, name="RdYlGn"),
   addCoef.col = "black", # Add coefficient of correlation
         tl.col="black", tl.srt=45 #Text label color and rotation
         # Combine with significance
)


Intuitívne vnímame súvislosti medzi `sqft_living` a `sqft_living15`, `sqft_lot` a `sqft_lot15`, `sqft_above` a `sqft_living` resp. `sqft_living`. Tieto premenné odstránime a zobrazíme si korelačnú maticu tohto intuitívneho datasetu. 

Významnejšie hodnoty korelácie pozorujeme medzi dvojicami `price` a `grade` 0.68, `price` a `sqft_living15` 0.62, `bedrooms` a `bathrooms` 0.56, `grade` a `sqft_living15` 0.69.

Korektne sa na multikolinearitu pozrieme pomocou nástrojov regresnej analýzy.

In [ ]:
h_tr_intuit<- subset (h_train, select = -c(sqft_living,sqft_lot,sqft_above))

In [ ]:
corrplot(cor(h_tr_intuit),
  method = "color",
  type = "upper", # show only upper side
   col=brewer.pal(n=8, name="PuOr"),
   addCoef.col = "black", # Add coefficient of correlation
         tl.col="black", tl.srt=45 #Text label color and rotation
         # Combine with significance
)


Najprv sa pokúsime vytvoriť model pomocou všetkých premenných. V tomto prípade funkcia lm() nestanoví jeden koeficient kvôli výskytu singularít v dátach - koeficient pre `sqft_basement` je NaN. Keď preusporiadame premenné v modeli, vypadnú nám koeficienty pre `sqft_above`. Funkcia VIF() na takýto model odpovedá chybovou hláškou *there are aliased coefficients in the model*. Dohromady máme pomerne spoľahlivý indikátor na multikolinearitu. 

Odstránime teda premennú `sqft_above` v súlade s našou intuitívnou úvahou.

In [ ]:
#lm(Y ~ .,data=Z)
m_all=lm(price~.,data = h_train)
summary(m_all)

In [ ]:
summary(lm(h_train$price~h_train$bedrooms + h_train$floors + h_train$bathrooms + h_train$view + h_train$waterfront + h_train$sqft_living  + h_train$grade + h_train$condition + h_train$sqft_basement + h_train$sqft_living15 + h_train$yr_renovated + h_train$yr_built + h_train$sqft_lot15 + h_train$sqft_above + h_train$sqft_lot))


In [ ]:
#vif(m_all)     #there are aliased coefficients in the model

Zredukovaný model disponuje všetkými koeficientmi. Vo výpise VIF() tohto modelu vyčnieva hodnota `sqft_living`, ktorá činí 5.4 a je najvyššia oproti ostatným. Stále však neprekročila hranicu *10* a teda je ešte prípustná.

Napriek tomu v ďalšom kroku prejdeme k intuitívnemu modelu, v ktorom budeme brať do úvahy hodnoty obytnej plochy a plochy pozemku z roku 2015. (Rozdiel medzi `sqft_xx` a `sqft_xx15` nie je z popisu dát jednoznačný. V riešení ho ponímame ako plochu v pôvodných plánoch domu a plochu aktuálnejšieho dáta 2015)



In [ ]:
h_tr_red=subset(h_train, select = -c(sqft_above))

In [ ]:
m_red=lm(price~.,data = h_tr_red)
summary(m_red)

In [ ]:
vif(m_red,data=h_tr_red)

In [ ]:
corr_kappa_red <- round(cor(h_tr_red), 1)
print('Kappa hodnota:')
kappa(corr_kappa_red,exact=T)

V intuitívne redukovanom modeli nám o niečo poklesla hodnota $R^2$ z $0.6692$ na $0.6633$. Poklesli hodnoty funkcie VIF() zostatkových premenných. Hodnota Kappa pre model bez plochy nadzemia činí 46%, teda *moderate strength of agreement*, kdežto pre intuitívne aditívne vylúčenie plánových plôch je Kappa hodnote 25% *fair agreement*.

In [ ]:
m_all_int=lm(price~.,data = h_tr_intuit)
summary(m_all_int)

In [ ]:
vif(m_all_int)

In [ ]:
corr_kappa_int <- round(cor(h_tr_intuit), 1)
print('Kappa hodnota:')
kappa(corr_kappa_int,exact=T)

In [ ]:
h_tr_not_liv<- subset (h_train, select = -c(sqft_living,sqft_above))
m_2=lm(price~.,data = h_tr_not_liv)
summary(m_2)
vif(m_2)
corr_kappa_2 <- round(cor(h_tr_not_liv), 1)
print('Kappa hodnota:')
kappa(corr_kappa_2,exact=T)

* Otázka 05
    
  Pouze pomocí trénovacích dat (tj., split == "train") a všech vybranných proměnných najděte vhodný lineární regresní model, který má za úkol predikovat co nejlépe cenu, tj. minimalizovat střední kvadratickou chybu (MSE). U výsledného modelu porovnejte VIF a Kappa s původní celkovou maticí regresorů.
  



* Otázka 06

   Pro Vámi vybraný model z předešlé otázky spočtěte příslušné infulenční míry. Uveďtě id pro 20 pozorování s největší hodnotou DIFF, největší hodnotou leverage (hatvalues) a největší hodnotou Cookovy vzdálenosti. (tj, 3 krát 20 hodnot). Jaká pozorování považujete za vlivná a odlehlá pozorování.


* Otázka 07

  Validujte model pomocí grafického znázornění reziduí (Residual vs Fitted, QQ-plot, Cookova vzdálenost, Leverages, ...).
  Identifikovali jste na základě této a předchozí otázky v datech nějaká podezřelá pozorování, která mohla vzniknout při úpravě (přepisu) datasetu? Doporučili byste tato pozorování z dat odstranit?

## Train, test, validation ...:  (možno vynechat pokud si jste jisti počtem bodů na zápočet)

* Otázka 08

  Pokud jste se rozhodli z dat odstranit nějaká pozorování, tak dále pracujtes s vyfiltrovaným datasetem a přetrénujte model z otázky 5. A spočtěte pro tento model $R^2$ statistiku a MSE jak na trénovacích tak testovacích datech (split == "test"). 

* Otázka 09

  Pomocí hřebenové regrese (případně pomocí LASSO a Elastic Net) zkuste najít nejlepší hyperparametr(y) tak, aby výsledný model měl co nejmenší MSE na testovacích datech.  

* Otázka 10

  Vyberte výsledný model a porovnejte MSE a $R^2$ na trénovacích, testovacích a validačních datech. Co z těchto hodnot usuzujete o kvalitě modelu a případném přetrénování? Je váš model vhodný pro predikci cen nemovitostí v okolí King County? Pokud ano, má tato predikce nějaká omezeni?